# Module 1 - LLM Zoomcamp

In [4]:
import os

from openai import OpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [25]:
load_dotenv()

True

In [5]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
     {
        "role": "user",
        "content": "List 5 tools or libraries to do High frequency trading"
    }   
    ]
)

In [13]:
print(response.choices[0].message.content)

High-frequency trading (HFT) requires sophisticated tools and libraries to handle large volumes of data, execute trades with low latency, and maintain reliable performance. Here are five tools and libraries that are commonly used in the HFT space:

1. **FIX Protocol Libraries:**
   - The Financial Information eXchange (FIX) protocol is a standard messaging protocol for the real-time exchange of securities transactions. Libraries like QuickFIX (available in C++, Java, Python, and .NET) are frequently used in HFT systems to communicate with trading venues.

2. **Quantlib:**
   - QuantLib is an open-source library for quantitative finance. It offers tools for modeling, trading strategy development, and risk management, although it is more commonly used for derivatives pricing and risk analytics than for HFT itself.

3. **KDB+/q:**
   - KDB+ is a high-performance time-series database used extensively in HFT. It is known for its lightning-fast data retrieval and ability to handle large data

## RAG - Retrieval Augmented Generation

In [13]:
from elasticsearch import Elasticsearch
import json
from tqdm.auto import tqdm

In [6]:
es_client = Elasticsearch("http://localhost:9200")

In [7]:
es_client.info()

ObjectApiResponse({'name': '628c37060246', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'uhL8fz6vRcmsRJgIZ4b9Lg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
_index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=_index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
with open("./documents.json") as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:01<00:00, 513.60it/s]


In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from elasticsearch import Elasticsearch


class SimpleRAG:

    def __init__(self, model) -> None:
        self.model = model
        self.es_client = Elasticsearch("http://localhost:9200")
        self.prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
                    Use only the facts from the CONTEXT when answering the QUESTION.
                    CONTEXT: 
                    {context}
                    """,
                ),
                ("human", "{question}"),
            ]
        )

    def elastic_search(self, query: str):
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "data-engineering-zoomcamp"
                        }
                    }
                }
            }
        }
    
        response = es_client.search(index=index_name, body=search_query)
        
        result_docs = []
        
        for hit in response['hits']['hits']:
            result_docs.append(hit['_source'])
        
        return result_docs

    def query(self, question: str):
        search_results = self.elastic_search(question)
        chain = self.prompt | self.model
        response = chain.invoke(
            {
                "context": search_results, # elasticsearch 
                "question": question,
            }
        )
        # print(search_results)
        return response.content

## Using the RAG

In [41]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

es_rag = SimpleRAG(
    model=llm
)

In [42]:
BASE_QUERY = 'What do I need to know before taking the course?'

In [43]:
es_rag.query(question=BASE_QUERY)

'Before taking the course, you can start by installing and setting up all the dependencies and requirements, which include a Google Cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git. Additionally, it would be beneficial to look over the prerequisites and syllabus to ensure you are comfortable with the subjects covered in the course.'